In [ ]:
import cv2 as cv
import numpy as np
from glob import glob
from matplotlib import pyplot as plt
from bisect import bisect_left
from copy import copy, deepcopy
import os
from PIL import Image,ExifTags

<h2>Constants

In [ ]:
INIT_MIN_NUM_MATCHES = 50
RAY_ANGLE_TH = 2 #degrees
TERMINATE_TH = 30

<h2>UTILS

In [ ]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

<h2>Read Images

In [ ]:
datasetPath = "Data/dinoSparseRing/"
filesNames = glob(datasetPath+'*.png') #TODO add jpg also
filesNames = sorted(filesNames)
images = [cv.imread(file) for file in filesNames]

<h2>Read EXIF Info

In [ ]:
def compute_focal_length(imgIdx):
    ccdWidths = {
        "samsung SM-A307FN": 1.7#todo for test only #1/2.8"
    }
    #img = Image.open('tryexif2.jpg')
    img = Image.open(filesNames[imgIdx])
    exif = {
        ExifTags.TAGS[k]: v
        for k, v in img._getexif().items()
        if k in ExifTags.TAGS
    }
    print(exif)
    #Get Focal Length
    focalN, focalD = exif['FocalLength']
    focalLen = float(focalN)/float(focalD)

    imgWidth = exif['ExifImageWidth']
    imgHeight = exif['ExifImageHeight']
    #????????#TODO
    if imgWidth < imgHeight:
        imgWidth,imgHeight = imgHeight,imgWidth
    #Get CCD Width
    ccdWidth = 1.0
    make = exif['Make']
    model = exif['Model']
    #Search in the DB
    if (make + ' ' + model) in ccdWidths:
        ccdWidth = ccdWidths[make + ' ' + model]
        print("ccdWidth: ", ccdWidth)
    #????????#TODO
    else:
        fplaneN, fplaneD = exif['FocalPlaneXResolution']
        if fplaneN != 0:
            ccdWidth = 25.4*float(imgWidth)*float(fplaneD)/float(fplaneN)
            print("ccdWidth: ", ccdWidth)
        else:
            ccdWidth = 0

    if (imgWidth == 0 or imgHeight == 0 or focalN == 0 or ccdWidth == 0):
        print ("focal length can't be determined")
    #Get Focal Length in Pixels
    focalLen = imgWidth * (focalLen / ccdWidth)
    print("focalLen: ", focalLen)
    return focalLen

<h2>Intrinsic Matrix

In [ ]:
'''
imgHeight, imgWidth, _ = images[0].shape   
#focalLen =  1.2 * max(imgHeight, imgWidth)
focalLen = compute_focal_length(0)
pp = (imgWidth/2, imgHeight/2)
K = [[focalLen, 0, pp[0]],
     [0, focalLen, pp[1]],
     [0,     0,       1]]
'''
focalLen = 3310.400000
imgHeight, imgWidth, _ = images[0].shape 
pp = (316.730000, 200.550000)
'''
K = [[3310.400000, 0.0, 316.730000],
     [0.0, 3325.500000, 200.550000],
     [0.0,     0.0,       1.0]]
'''
K = [[3310.400000, 0.0, 316.730000],
     [0.0, 3310.400000, 200.550000],
     [0.0,     0.0,       1.0]]

<h2>SIFT

In [ ]:
def compute_sift():
    for imgIdx, img in enumerate(images):
        #show_images([img],["img"])
        #gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        sift = cv.xfeatures2d.SIFT_create()
        # find keypoints and descriptors with SIFT
        kp,des = sift.detectAndCompute(img, None)
        imageModel = {
            "keyPts": kp,
            "descriptors": des,
            "tracksFeaturesIdx": [],
            "projMat": np.zeros((3,4))
        }
        imagesModels.append(imageModel)
        print("Number of keypoints ",len(kp))
        #draw
        #imgKp = cv.drawKeypoints(gray,kp,copy(img),flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        #show_images([imgKp],["img"]) 

<h2>Sift BOOk

In [ ]:
'''
def process_image(imagename,resultname,params="--edge-thresh 10 --peak-thresh 5"):
    """ Process an image and save the results in a file. """
    if imagename[-3:] != 'pgm':
        # create a pgm file
        im = Image.open(imagename).convert('L')
        im.save('tmp/tmp.pgm')
        imagename = 'tmp/tmp.pgm'
    cmmd = str("sift "+imagename+" --output="+resultname+" "+params)
    os.system(cmmd)
    print("processed", imagename, "to", resultname)
    
def read_features_from_file(filename):
    """ Read feature properties and return in matrix form. """
    f = np.loadtxt(filename)
    return f[:,:4],f[:,4:] # feature locations, descriptors
def plot_features(im,locs,circle=False):
    """ Show image with features. input: im (image as array),
    locs (row, col, scale, orientation of each feature). """
    def draw_circle(c,r):
        t = arange(0,1.01,.01)*2*pi
        x = r*cos(t) + c[0]
        y = r*sin(t) + c[1]
        plot(x,y,'b',linewidth=2)
    imshow(im)
    if circle:
        for p in locs:
            draw_circle(p[:2],p[2])
    else:
        plot(locs[:,0],locs[:,1],'ob')
    axis('off')
    
#for i,img in enumerate(images):
for file in glob(datasetPath+'*.png'):
    img = np.array(Image.open(file).convert('L'))
    print(file)
    process_image(file,  str(i)+'.sift')
    l1,d1 = read_features_from_file(str(i)+'.sift')
    #figure()
    #gray()
    plot_features(img,l1,circle=True)
    #show()
'''

<h2> Matching

In [ ]:
def compute_matches(idx1,idx2): #image indices
    #Matching descriptors
    # FLANN parameters
    FLANN_INDEX_KDTREE = 0 #TODO try:1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(imagesModels[idx1]["descriptors"], imagesModels[idx2]["descriptors"], k=2)
    
    pts1 = []
    pts2 = []
    #Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.6*n.distance:
            pts2.append(imagesModels[idx2]["keyPts"][m.trainIdx].pt)
            pts1.append(imagesModels[idx1]["keyPts"][m.queryIdx].pt) 
            
            matchesIdx[idx1][idx2].append({'idx1': m.queryIdx, 'idx2':  m.trainIdx})
            matchesIdx[idx2][idx1].append({'idx1': m.trainIdx, 'idx2': m.queryIdx})
            matchesMask[i] = [1,0]
    
    #Draw Matches
    draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv.DrawMatchesFlags_DEFAULT)
    img3 = cv.drawMatchesKnn(images[idx1], imagesModels[idx1]["keyPts"], images[idx2], imagesModels[idx2]["keyPts"], matches,None,**draw_params)
    #show_images([images[idx1], images[idx2], img3], ["img1: idx " + str(idx1), "img2: idx "+str(idx2), "matches"])
    
    return pts1, pts2
    '''
    pts1 = np.float32(pts1)
    pts2 = np.float32(pts2)
    #print(pts1)
    #print(pts2)
    ;;;;;;;;;;;;;;;;;;;;;;;
    print("pts1.shape:%s\tpts2.shape:%s"%(pts1.shape,pts2.shape))
    fundmentalMat, mask = cv.findFundamentalMat(pts1,pts2,cv.FM_LMEDS)
    # We select only inlier points
    #pts1 = pts1[mask.ravel()==1]
    #pts2 = pts2[mask.ravel()==1]
    print(("Fundmental Matrix between image[%d] and image[%d]:\n%a") % (idx1,idx2,fundmentalMat))
    return fundmentalMat
    '''

<h2>Compute Fundamental Matrix

In [ ]:
def compute_fundamental(x1,x2):
    """ Computes the fundamental matrix from corresponding points
    (x1,x2 3*n arrays) using the normalized 8 point algorithm.
    each row is constructed as
    [x’*x, x’*y, x’, y’*x, y’*y, y’, x, y, 1] """
    n = x1.shape[1]
    if x2.shape[1] != n:
        raise ValueError("Number of points don’t match.")
    # build matrix for equations
    A = np.zeros((n,9))
    for i in range(n):
        A[i] = [x1[0,i]*x2[0,i], x1[0,i]*x2[1,i], x1[0,i]*x2[2,i],
        x1[1,i]*x2[0,i], x1[1,i]*x2[1,i], x1[1,i]*x2[2,i],
        x1[2,i]*x2[0,i], x1[2,i]*x2[1,i], x1[2,i]*x2[2,i] ]
    # compute linear least square solution
    U,S,V = np.linalg.svd(A)
    F = V[-1].reshape(3,3)
    # constrain F
    # make rank 2 by zeroing out last singular value
    U,S,V = np. linalg.svd(F)
    S[2] = 0
    F = np.dot(U,np.dot(np.diag(S),V))
    return F

In [ ]:
def compute_fundamental_normalized(x1,x2):
    """ Computes the fundamental matrix from corresponding points
    (x1,x2 3*n arrays) using the normalized 8 point algorithm. """
    n = x1.shape[1]
    if x2.shape[1] != n:
        raise ValueError("Number of points don’t match.")
    # normalize image coordinates
    x1 = x1 / x1[2]
    mean_1 = np.mean(x1[:2],axis=1)
    S1 = np.sqrt(2) / np.std(x1[:2])
    T1 = np.array([[S1,0,-S1*mean_1[0]],[0,S1,-S1*mean_1[1]],[0,0,1]])
    x1 = np.dot(T1,x1)
    x2 = x2 / x2[2]
    mean_2 = np.mean(x2[:2],axis=1)
    S2 = np.sqrt(2) / np.std(x2[:2])
    T2 = np.array([[S2,0,-S2*mean_2[0]],[0,S2,-S2*mean_2[1]],[0,0,1]])
    x2 = np.dot(T2,x2) 
    # compute F with the normalized coordinates
    F = compute_fundamental(x1,x2)
    # reverse normalization
    F = np.dot(np.transpose(T1),np.dot(F,T2))
    return F/F[2,2]

In [ ]:
class RansacModel(object):
    """ Class for fundmental matrix fit with ransac.py from
    http://www.scipy.org/Cookbook/RANSAC"""
    def __init__(self,debug=False):
        self.debug = debug
        
    def fit(self,data):
        """ Estimate fundamental matrix using eight
        selected correspondences. """
        # transpose and split data into the two point sets
        data = np.transpose(data)
        x1 = data[:3,:8]
        x2 = data[3:,:8]
        # estimate fundamental matrix and return
        F = compute_fundamental_normalized(x1,x2)
        return F
    
    def get_error(self,data,F):
        """ Compute x^T F x for all correspondences,
        return error for each transformed point. """
        # transpose and split data into the two point
        data = np.transpose(data)
        x1 = data[:3]
        x2 = data[3:]
        # Sampson distance as error measure
        Fx1 = np.dot(F,x1)
        Fx2 = np.dot(F,x2)
        denom = Fx1[0]**2 + Fx1[1]**2 + Fx2[0]**2 + Fx2[1]**2
        err = ( np.diag(np.dot(np.transpose(x1),np.dot(F,x2))) )**2 / denom
        # return error per point
        return err
        

In [ ]:
def F_from_ransac(x1,x2,model,maxiter=5000,match_theshold=1e-6):
    """ Robust estimation of a fundamental matrix F from point
    correspondences using RANSAC (ransac.py from
    http://www.scipy.org/Cookbook/RANSAC).
    input: x1,x2 (3*n arrays) points in hom. coordinates. """
    import ransactmp as ransac
    data = np.vstack((x1,x2))
    # compute F and return with inlier index
    F,ransac_data = ransac.ransac(np.transpose(data),model,8,maxiter,match_theshold,20,debug = False,return_all=True)
    return F, ransac_data['inliers']

<h2> Tracks

In [ ]:
class KeyList(object):
    # bisect doesn't accept a key function, so we build the key into our sequence.
    def __init__(self, l, key):
        self.l = l
        self.key = key
    def __len__(self):
        return len(self.l)
    def __getitem__(self, index):
        return self.key(self.l[index])
    

def get_matched_idx(matchesList, searchItem):
    i = bisect_left(KeyList(matchesList, key=lambda k: k['idx1']), searchItem) 
    if i != len(matchesList) and matchesList[i]['idx1'] == searchItem: return matchesList[i]['idx2']
    else: return -1
    
#Test 
'''
matchesList = [{'idx1': 100, 'idx2': 5}, {'idx1': 2, 'idx2': 88}, {'idx1': 7, 'idx2': 5}, {'idx1': 9, 'idx2': 10}]
print(matchesList[0]['idx1'])
matchesList = sorted(matchesList, key=lambda k: k["idx1"]) 
print(matchesList[0]['idx1'])
print(get_matched_idx(matchesList, 2))
'''

In [ ]:
def compute_tracks():
    #Sort matchesIdx #TODO #find faster way 
    for i in range(len(images)):
        for j in range(len(images)):
            matchesIdx[i][j] =  sorted(matchesIdx[i][j], key=lambda k: k["idx1"])

    for imgIdx in range(len(images)):
        if len(matchesIdx[imgIdx]) == 0: continue #??TODO neighbors
        flags = np.zeros(len(imagesModels[imgIdx]['keyPts']))
        imagesModels[imgIdx]['keyFlags'] = flags

    for imgIdx in range(len(images)):
        if len(matchesIdx[imgIdx]) == 0: continue #??TODO neighbors
        '''
        BFS from each feature
        images --> Nodes
        Features --> Egdes
        '''
        for featureIdx in range(len(imagesModels[imgIdx]['keyPts'])):
            #if the feature was visited
            if imagesModels[imgIdx]['keyFlags'][featureIdx] == 1: continue
            featuresTrack = []
            featuresQueue = []
            imageVisited = np.zeros(len(images))
            #Mark the feature as visited
            imagesModels[imgIdx]['keyFlags'][featureIdx] = 1
            featuresTrack.append((imgIdx, featureIdx))
            featuresQueue.append((imgIdx, featureIdx))
            imageVisited[imgIdx] = 1
            while not len(featuresQueue) == 0:
                feature = featuresQueue.pop(0)
                parImgIdx = feature[0]
                parFeatureIdx = feature[1]
                for childIdx in range(len(matchesIdx[parImgIdx])): #??TODO neighbors
                    if imageVisited[childIdx] == 1: continue
                    childFeatureIdx = get_matched_idx(matchesIdx[parImgIdx][childIdx], parFeatureIdx)# search for parFeatureIdx get the corresponding idx
                    if childFeatureIdx == -1: continue
                    if imagesModels[childIdx]['keyFlags'][childFeatureIdx] == 1: continue
                    imagesModels[childIdx]['keyFlags'][childFeatureIdx] = 1
                    featuresTrack.append((childIdx, childFeatureIdx))
                    featuresQueue.append((childIdx, childFeatureIdx))
                    imageVisited[childIdx] = 1
            if len(featuresTrack) >= 2:
                tracks.append(featuresTrack)
    for trackIdx in range(len(tracks)):
        for imgIdx,featureIdx in tracks[trackIdx]:
            imagesModels[imgIdx]['tracksFeaturesIdx'].append((trackIdx, featureIdx))

<h2>Projection Matrix For initialization

In [ ]:
#cam2 is the ref #TODO make sure?
#tODO test on each pair of images
def proj_matrix_for_initial_camera(imgIdx1, imgIdx2):
    #TODO instead of the two for loops --> Store the value of pts
    matchIndices1 = []
    matchIndices2 = []
    pts1 = []
    pts2 = []
    for item in matchesIdx[imgIdx1][imgIdx2]:
        matchIndices1.append(item['idx1'])
        matchIndices2.append(item['idx2'])
    
    for i in range(len(matchIndices1)):
        pts1.append(imagesModels[imgIdx1]['keyPts'][matchIndices1[i]].pt)
        pts2.append(imagesModels[imgIdx2]['keyPts'][matchIndices2[i]].pt)

    E, mask = cv.findEssentialMat(np.array(pts1), np.array(pts2), 
                             focal = focalLen,
                             pp = (imgWidth/2, imgHeight/2), #TODO may be 0,0 or imgWidth/2, imgHeight/2
                             method = cv.RANSAC, 
                             prob = 0.999, 
                             threshold = 1.0) 

    '''
   ** threshold – Parameter used for RANSAC. 
    It is the maximum distance from a point to an epipolar line in pixels, beyond which the point is considered an outlier
    and is not used for computing the final fundamental matrix.
    It can be set to something like 1-3, depending on the accuracy of the point localization, image resolution, and the image noise.
   ** prob – Parameter used for the RANSAC or LMedS methods only. 
    It specifies a desirable level of confidence (probability) that the estimated matrix is correct
    '''
    #retval:NumOfInliers
    retval, R, t, mask, triangulatedPoints = cv.recoverPose(E, np.array(pts1), np.array(pts2), np.array(K), distanceThresh = 50) #TODO  # 2pass l mask l tal3 mn l essential?
    return R, t

<h2>Projection Matrix From point correspondence Book
    

In [ ]:
def compute_P(x,X):
    """ Compute camera matrix from pairs of
    2D-3D correspondences (in homog. coordinates). """
    n = x.shape[1]
    if X.shape[1] != n:
        raise ValueError("Number of points don’t match.")
        
    # create matrix for DLT solution
    M = np.zeros((3*n,12+n))
    for i in range(n):
        M[3*i,0:4] = X[:,i]
        M[3*i+1,4:8] = X[:,i]
        M[3*i+2,8:12] = X[:,i]
        M[3*i:3*i+3,i+12] = -x[:,i]
    U,S,V = np.linalg.svd(M)
    return V[-1,:12].reshape((3,4))

"""
#Test
# load 2D points for each view to a list
points2D = [np.loadtxt('Data/dataset_camera_3_MertonCollegeI/2D/00'+str(i+1)+'.corners').T for i in range(3)]
# load 3D points
points3D = np.loadtxt('Data/dataset_camera_3_MertonCollegeI/3D/p3d').T
# load correspondences
corr = np.genfromtxt('Data/dataset_camera_3_MertonCollegeI/2D/nview-corners',dtype='int',missing_values='*')
# load cameras to a list of Camera objects
P = [np.loadtxt('Data/dataset_camera_3_MertonCollegeI/2D/00'+str(i+1)+'.P') for i in range(3)]

corr = corr[:,0] # view 1
ndx3D = np.where(corr>=0)[0] # missing values are -1
ndx2D = corr[ndx3D]
# select visible points and make homogeneous
x = points2D[0][:,ndx2D] # view 1
x = np.vstack( (x,np.ones(x.shape[1])) )
X = points3D[:,ndx3D]
X = np.vstack( (X,np.ones(X.shape[1])) )
print("x: ", x.shape, "X: ", X.shape)
print("x: ", x, "X: ", X)
#### estimate P ####
Pest = compute_P(x,X)
print(Pest/ Pest[2,3])
print(P[0] / P[0][2,3])
###project the  estimated 3D point  #Try
#the true points in blue and the estimated camera projection in red.
xest = Pest.project(X)
'''
# plotting
figure()
imshow(im1)
plot(x[0],x[1],’b.’)
plot(xest[0],xest[1],’r.’)
axis(’off’)
show()
'''
"""

<h2>Bundle Adjustment_implementation

In [ ]:
"""
def reprojectionError(camMatrices, X_est, imgCoords):
    '''
    Inputs: camMatrices is a numCameras by 3 by 4 numpy array.
    X_est is a 3 by numPoints numpy array.
    imgCoords is a 2 by numPoints numpy array.
    '''
    
    numCameras = camMatrices.shape[0]
    numPoints = X_est.shape[1]
    
    proj = camMatrices @ np.vstack((X_est,np.ones((1,numPoints))))
    cost = np.sum((proj[:,0,:]/proj[:,2,:] - imgCoords[:,0,:])**2 + (proj[:,1,:]/proj[:,2,:] - imgCoords[:,1,:])**2)
            
    return cost
"""

In [ ]:
"""
def bundleAdjust_LM(camMatrices, X_est, imgCoords):
    '''
    We perform bundle adjustment using the Levenberg-Marquardt algorithm.
    See bundleAdjust_sparseLM for the sparse version of this algortihm.
    
    Inputs: camMatrices is a numCameras by 3 by 4 numpy array.
    X_est is a 3 by numPoints numpy array.
    imgCoords is a numCameras by 2 by numPoints numpy array.
    '''
    
    lmbda = .0001
    maxIter = 10
    reduceFactor = .1
    increaseFactor = 10
    
    numCameras = camMatrices.shape[0]
    numPoints = X_est.shape[1]
    
    d = 12*numCameras + 3*numPoints
    
    P = camMatrices.copy()
    X = X_est.copy()
    
    reprojErr = reprojectionError(P,X,imgCoords)
    reprojErrors = np.zeros(maxIter)
    
    for iter in range(maxIter):
        
        print('iter is: ', iter)
        
        # Set up the normal equations.
        mtrx = np.zeros((d,d))
        rhs = np.zeros(d)
        
        for i in range(numCameras):
            
            Pi = P[i]
            rows = np.zeros((12, d))
            
            for j in range(numPoints):
                
                Xj = X[:,j]
                x_ij = imgCoords[i,:,j]
                
                dxHatdP_ij = dxHatdP(Pi,Xj)
                dxHatdX_ij = dxHatdX(Pi,Xj)
                
                rows[:,12*i:12*i+12] += dxHatdP_ij.T @ dxHatdP_ij
                rows[:,12*numCameras + 3*j:12*numCameras + 3*j + 3] = dxHatdP_ij.T @ dxHatdX_ij
                
                rhs[12*i:12*i+12] += dxHatdP_ij.T @ (x_ij - xHat(Pi,Xj))
                
            
            rows[:,12*i:12*i+12] += lmbda * np.eye(12)
            
            mtrx[12*i:12*i+12,:] = rows
            
        for j in range(numPoints):
            
            Xj = X[:,j]
            rows = np.zeros((3,d))
            
            for i in range(numCameras):
                
                Pi = P[i]
                x_ij = imgCoords[i,:,j]
                dxHatdP_ij = dxHatdP(Pi,Xj)
                dxHatdX_ij = dxHatdX(Pi,Xj)
                
                rows[:,12*i:12*i+12] = dxHatdX_ij.T @ dxHatdP_ij
                rows[:,12*numCameras + 3*j:12*numCameras + 3*j+3] += dxHatdX_ij.T @ dxHatdX_ij
                
                rhs[12*numCameras + 3*j:12*numCameras + 3*j + 3] += dxHatdX_ij.T @ (x_ij - xHat(Pi,Xj))
                
            rows[:,12*numCameras + 3*j:12*numCameras + 3*j+3] += lmbda * np.eye(3)
            
            mtrx[12*numCameras + 3*j:12*numCameras + 3*j + 3] = rows
            
        # Now solve normal equations
        vec = np.linalg.solve(mtrx, rhs)
        
        dP = vec[0:12*numCameras]
        dX = vec[12*numCameras:]
        
        dP = np.reshape(dP,(numCameras,3,4))
        dX = np.reshape(dX,(numPoints,3)).T

        P_next = P + dP
        X_next = X + dX
        
        reprojErr_next = reprojectionError(P_next, X_next, imgCoords)
        
        if reprojErr_next < reprojErr:
            
            P = P_next
            X = X_next
            reprojErr = reprojErr_next
            lmbda *= reduceFactor

        else:
            lmbda *= increaseFactor
            reprojErrors[iter] = reprojErr
            
        reprojErrors[iter] = reprojErr
        print('iter: ', iter, 'reprojError: ', reprojErr_next, 'lmbda: ', lmbda)
        
    return P, X, reprojErrors
"""

In [ ]:
"""
def bundleAdjust_sparseLM(camMatrices, X_est, imgCoords):
    '''
    We perform bundle adjustment using the sparse Levenberg-Marquardt algorithm.
    
    Inputs: camMatrices is a numCameras by 3 by 4 numpy array.
    X_est is a 3 by numPoints numpy array.
    imgCoords is a numCameras by 2 by numPoints numpy array.
    '''
    
    lmbda = .0001
    maxIter = 10
    reduceFactor = .1
    increaseFactor = 10
    
    numCameras = camMatrices.shape[0]
    numPoints = X_est.shape[1]
    
    d = 12*numCameras + 3*numPoints
    
    P = camMatrices.copy()
    X = X_est.copy()
    
    reprojErr = reprojectionError(P,X,imgCoords)
    reprojErrors = np.zeros(maxIter)
    
    for iter in range(maxIter):
        
        print('iter is: ', iter)
        
        # Set up the normal equations.
        
        # upperLeftBlocks = np.zeros((numCameras,12,12))
        # coefficient matrix for normal equations is partitioned into
        # upper left (UL), upper right (UR), lower left (LL), and lower right (LR) blocks.
        # Lower left block is transpose of upper right block.
        # The lower right matrix is itself block diagonal.
        UL = np.zeros((12*numCameras,12*numCameras))
        UR = np.zeros((12*numCameras,3*numPoints))
        LR_blocks= np.zeros((numPoints,3,3))
        rhs = np.zeros(d) # right hand side of normal equations.
        
        for i in range(numCameras):
            
            Pi = P[i]
            rows = np.zeros((12, d))
            
            for j in range(numPoints):
                
                Xj = X[:,j]
                x_ij = imgCoords[i,:,j]
                
                dxHatdP_ij = dxHatdP(Pi,Xj)
                dxHatdX_ij = dxHatdX(Pi,Xj)
                
                rows[:,12*i:12*i+12] += dxHatdP_ij.T @ dxHatdP_ij
                rows[:,12*numCameras + 3*j:12*numCameras + 3*j + 3] = dxHatdP_ij.T @ dxHatdX_ij
                
                UL[12*i:12*i+12,12*i:12*i+12] += dxHatdP_ij.T @ dxHatdP_ij
                UR[12*i:12*i+12,3*j:3*j + 3] = dxHatdP_ij.T @ dxHatdX_ij
                
                rhs[12*i:12*i+12] += dxHatdP_ij.T @ (x_ij - xHat(Pi,Xj))
                
            UL[12*i:12*i+12,12*i:12*i+12] += lmbda * np.eye(12)
            
        for j in range(numPoints):
            
            Xj = X[:,j]
            
            for i in range(numCameras):
                
                Pi = P[i]
                x_ij = imgCoords[i,:,j]
                dxHatdP_ij = dxHatdP(Pi,Xj)
                dxHatdX_ij = dxHatdX(Pi,Xj)
                
                LR_blocks[j] += dxHatdX_ij.T @ dxHatdX_ij
                
                rhs[12*numCameras + 3*j:12*numCameras + 3*j + 3] += dxHatdX_ij.T @ (x_ij - xHat(Pi,Xj))
                
            LR_blocks[j] += lmbda * np.eye(3)
            
        rhs_upper = rhs[0:12*numCameras]
        rhs_lower = rhs[12*numCameras:]
        
        LR_blocks_inv = np.zeros(LR_blocks.shape)
        LR_inv_times_rhs_lower = np.zeros(rhs_lower.shape)
        LR_inv_times_LL = np.zeros(UR.T.shape)
        for j in range(numPoints):
            LR_blocks_inv[j] = np.linalg.inv(LR_blocks[j])
            LR_inv_times_rhs_lower[3*j:3*j+3] = LR_blocks_inv[j] @ rhs_lower[3*j:3*j+3]
            LR_inv_times_LL[3*j:3*j+3] = LR_blocks_inv[j] @ UR.T[3*j:3*j+3,:]
            
        dP = np.linalg.solve(UL- UR @ LR_inv_times_LL, rhs_upper - UR @ LR_inv_times_rhs_lower)
        dX = LR_inv_times_rhs_lower - LR_inv_times_LL @ dP
        
        dP = np.reshape(dP,(numCameras,3,4))
        dX = np.reshape(dX,(numPoints,3)).T

        P_next = P + dP
        X_next = X + dX
        
        reprojErr_next = reprojectionError(P_next, X_next, imgCoords)
        
        if reprojErr_next < reprojErr:
            
            P = P_next
            X = X_next
            reprojErr = reprojErr_next
            lmbda *= reduceFactor

        else:
            lmbda *= increaseFactor
            reprojErrors[iter] = reprojErr
            
        reprojErrors[iter] = reprojErr
        print('iter: ', iter, 'reprojError: ', reprojErr_next, 'lmbda: ', lmbda)
        
    return P, X, reprojErrors
"""

In [ ]:
def get_intrinsic_matrix():
    return K

In [ ]:
#TODO test
def get_reference_proj(): 
    KR = np.matmul(get_intrinsic_matrix(),np.eye(3, dtype=int))
    proj = np.zeros((3,4))
    proj[:,:-1] = KR
    return proj

def compute_proj(R,t):
    KR = np.matmul(get_intrinsic_matrix(),R)
    return np.append(KR, t, axis=1) #TODO change np.append

In [ ]:
def factorize_proj_matrix(P):
    """ Factorize the camera matrix into K,R,t as P = K[R|t]. """
    # factor first 3*3 part
    R,K1 = np.linalg.qr(P[:,:3])
    # make diagonal of K positive
    T = np.diag(np.sign(np.diag(K1)))
    if np.linalg.det(T) < 0:
        T[1,1] *= -1
    K1 = np.dot(K1,T)
    R = np.dot(T,R) # T is its own inverse
    t = np.dot(np.linalg.inv(K1),P[:,3])
    return K1, R, t.reshape(3,1)

"""
#test
P = get_reference_proj()
print(factorize_proj_matrix(P))
"""

In [ ]:
#TODO ADD projmat = zeroes initialization
def compute_ray_angle(pt1,pt2,imgIdx1,imgIdx2):
    #k1,R1,t1,_,_,_,_ = cv.decomposeProjectionMatrix(imagesModels[imgIdx1]['projMat'])
    #k2,R2,t2,_,_,_,_ = cv.decomposeProjectionMatrix(imagesModels[imgIdx2]['projMat'])
    k1, R1,t1 = factorize_proj_matrix(imagesModels[imgIdx1]['projMat'])
    k2,R2,t2 = factorize_proj_matrix(imagesModels[imgIdx2]['projMat'])
    
    #Should have the same K --> check
    #print("[check] k1:\n", k1, "k2:\n", k2)
    pt1Norm = np.matmul(np.linalg.inv(k1), pt1.reshape(3,1))
    pt2Norm = np.matmul(np.linalg.inv(k2), pt2.reshape(3,1))
    pt1Norm = pt1Norm/pt1Norm[2]
    pt2Norm = pt2Norm/pt2Norm[2]
    pt1Norm[2] = -1 #t
    pt2Norm[2] = -1
    
    Rtrans1 = np.transpose(R1)
    Rtrans2 = np.transpose(R2)
    
    Rpt1 = np.matmul(Rtrans1, pt1Norm)
    Rpt2 = np.matmul(Rtrans2, pt2Norm)
    
    pW1 = Rpt1 + t1
    pW2 = Rpt2 + t2
    
    ray1 = pW1 - t1
    ray2 = pW2 - t2
    dot = np.dot(np.squeeze(ray2), np.squeeze(ray1))
    
    magnitude = np.linalg.norm(ray1)*np.linalg.norm(ray2)
    angle = np.math.acos(dot/magnitude)
    angle = np.math.degrees(angle)
    return angle

In [ ]:
#help(cv.decomposeProjectionMatrix)

<h2>Main

In [ ]:
imagesModels = []
matchesIdx = [[list() for x in range(len(images))] for y in range(len(images))] 
tracks = []
print("SIFT")
compute_sift()
print(len(imagesModels))
#matchScores = np.zeros((len(images), len(images))) #TODO compute from it the INIT_MIN_NUM_MATCHES
homographyScores = []
#model = RansacModel()
for i in range(len(images)):
    for j in range(i+1, len(images)):
        pts1, pts2 = compute_matches(i,j)
        #matchScores[i][j] = len(pts1)
        #matchScores[j][i] = len(pts1)
        #print("No. of matched features between ", str(i) , "&", str(j),len(pts1))
        '''
        #Compute Fundamental matrix
        #Add "t" to every point
        pts1Homogenous = cv.convertPointsToHomogeneous(np.float32(pts1)).reshape((-1,3))
        pts2Homogenous = cv.convertPointsToHomogeneous(np.float32(pts2)).reshape((-1,3))
        pts1Homogenous = np.transpose(pts1Homogenous)
        pts2Homogenous = np.transpose(pts2Homogenous)
        if(len(pts1) < 8):
            print("Couldn't find good 8 matches to compute Fundamental matrix")
        else:
            F,inliers = F_from_ransac(pts1Homogenous,pts2Homogenous,model,maxiter=2048,match_theshold=1)
            print("Fundamental Matrix between ", str(i) , "&", str(j), F)
            #tODO refine matches
        '''
        
        pts1 = np.float32(pts1).reshape(-1,1,2)
        pts2 = np.float32(pts2).reshape(-1,1,2)
        if len(pts1) < 4:
            print("Couldn't find good 4 matches to compute Homograhy matrix")
        else:  
            H, mask = cv.findHomography(pts1, pts2, cv.RANSAC, 0.004*max(imgHeight, imgWidth))
            HinliersNum = list(mask).count(1)
            #print("HinliersNum: ", HinliersNum)
            if len(pts1) >= INIT_MIN_NUM_MATCHES:
                homographyScores.append({"idx1":i, "idx2": j, "percentage": HinliersNum/len(pts1)})
                
compute_tracks()
print("tracks: ", tracks)
#print("number of tracks: ", len(tracks))
homographyScores =  sorted(homographyScores, key=lambda k: k["percentage"])
#print("homographyScores: ", homographyScores)
img1Idx = homographyScores[0]['idx1']
img2Idx = homographyScores[0]['idx2']
R, t = proj_matrix_for_initial_camera(img1Idx, img2Idx)
#TODO Assume: img1Idx is the ref
imagesModels[img1Idx]['projMat'] = get_reference_proj()
imagesModels[img2Idx]['projMat'] = compute_proj(R,t)
print("initial images: ", img1Idx, " & ", img2Idx)
print("1st two images --> Done")
#get tracks visible in the two images
#print("img1Idx: ", img1Idx, "img2Idx: ", img2Idx)
#print("imagesModels[img1Idx]['tracksFeaturesIdx']: ", imagesModels[img1Idx]['tracksFeaturesIdx'])
dicTracksFeaturesIdx1 = {k:v for k, v in imagesModels[img1Idx]['tracksFeaturesIdx']}
dicTracksFeaturesIdx2 = {k:v for k, v in imagesModels[img2Idx]['tracksFeaturesIdx']}
#print("dicTracksFeaturesIdx1: ",dicTracksFeaturesIdx1, "dicTracksFeaturesIdx2: ", dicTracksFeaturesIdx2)
pts1 = []
pts2 = []
usedTracks = []
usedImagesIdx = []
usedImagesIdx.append(img1Idx) #TODO may remove it from images/deepcopy of images and remove it from there to modify the outer loop of add new camera
usedImagesIdx.append(img2Idx)
viewPts = []
pts3D = []
track3D = [None] * len(tracks)
for track in dicTracksFeaturesIdx1.keys() & dicTracksFeaturesIdx2.keys():
    usedTracks.append(track)
    featureIdx1 = dicTracksFeaturesIdx1[track]
    featureIdx2 = dicTracksFeaturesIdx2[track]
    pt1 = imagesModels[img1Idx]['keyPts'][featureIdx1].pt
    pt2 = imagesModels[img2Idx]['keyPts'][featureIdx2].pt
    pts1.append(pt1)#TODO may be not needed
    pts2.append(pt2)#TODO may be not needed
    viewPts.append([(img1Idx,pt1), (img2Idx,pt2)])
    
    #triangulate tracks visible in the two images
    triangulatedPointsHomogeneous = cv.triangulatePoints(imagesModels[img1Idx]["projMat"],imagesModels[img2Idx]["projMat"],np.array(pt1),np.array(pt2))
    pt3D = triangulatedPointsHomogeneous[:4, :] / triangulatedPointsHomogeneous[3, :] #divide by t
    track3D[track] = pt3D
    #pts3D.extend(deepcopy(triangulatedPointsHomogeneous)) #TODO deepcopy is needed                                               
    pts3D.append(pt3D) #TODO deepcopy is needed?
'''
TWo frame Bundle adjustment #here
'''
"""
camMatricesEst = [imagesModels[img1Idx]['projMat'], imagesModels[img2Idx]['projMat']]
XEst = []
imgCoords = [pts1, pts2]
PAdjusted, ptsX, reprojErrors = bundleAdjust_sparseLM(camMatricesEst, XEst, imgCoords)
imagesModels[img1Idx]['projMat'] = 
imagesModels[img2Idx]['projMat'] = 
"""
'''
Add New Camera
'''
terminate = False #here to add TERMINATE_TH if condition
while len(usedImagesIdx) < len(imagesModels) and not terminate:
    #get the camera that observes the largest number of usedTracks
    maxObservedTracksCnt = 0
    selectedImgIdx = -1
    ptsTmp = []
    selectedpts3DTmp = []
    pts = []
    for imgIdx in range(len(images)):
        if imgIdx in usedImagesIdx:
            continue
        observedTracksCnt = 0
        ptsTmp = []
        selectedpts3DTmp = []
        dicTracksFeaturesIdx = {k:v for k, v in imagesModels[imgIdx]['tracksFeaturesIdx']}
        for track in dicTracksFeaturesIdx.keys() & usedTracks: 
            observedTracksCnt += 1
            featureIdx = dicTracksFeaturesIdx[track]
            ptsTmp.append((imagesModels[imgIdx]['keyPts'][featureIdx].pt[0],imagesModels[imgIdx]['keyPts'][featureIdx].pt[1],1))
            selectedpts3DTmp.append(track3D[track])
        if observedTracksCnt > maxObservedTracksCnt:
            maxObservedTracksCnt = observedTracksCnt
            selectedImgIdx = imgIdx
            pts = deepcopy(ptsTmp) #todo deepcopy needed?
            selectedpts3D = deepcopy(selectedpts3DTmp) #todo deepcopy needed?
    if selectedImgIdx != -1:
        #print("parameters: ", np.transpose(np.array(pts)),np.transpose(np.array(selectedpts3D)))
        #print("shape: ", np.transpose(np.array(pts)).shape, np.transpose(np.squeeze(np.array(selectedpts3D))).shape)
        imagesModels[selectedImgIdx]['projMat'] = compute_P(np.transpose(np.array(pts)),np.transpose(np.squeeze(np.array(selectedpts3D))))#TODO check dimensions
        #R,t,inliers = cv.solvePnPRansac(np.squeeze(np.array(selectedpts3D),np.array(pts),K) #todo hst5dem l inliers?
        #print("solvePnPRansac: R:", R, " t: ", t)
        usedImagesIdx.append(selectedImgIdx)
    #Add points observed by the new camera
    addedPt = None
    addedPt3D = None
    maxSeparationAngle = 0
    separationAngle = 0
    for trackIdx,featureIdx in imagesModels[selectedImgIdx]['tracksFeaturesIdx']:
        for imgIdx, featureIdx2 in tracks[trackIdx]:
            if imgIdx == selectedImgIdx:
                continue
            if imgIdx in usedImagesIdx and not trackIdx in usedTracks:
                pt1 = imagesModels[selectedImgIdx]['keyPts'][featureIdx].pt
                pt2 = imagesModels[imgIdx]['keyPts'][featureIdx2].pt
                triangulatedPointsHomogeneous = cv.triangulatePoints(imagesModels[selectedImgIdx]["projMat"],imagesModels[imgIdx]["projMat"],pt1,pt2)
                separationAngle = compute_ray_angle(np.array([pt1[0],pt1[1],1]),np.array([pt2[0],pt2[1],1]),selectedImgIdx,imgIdx)
                if separationAngle > maxSeparationAngle:
                    maxSeparationAngle = separationAngle  
                    addedPt3D = triangulatedPointsHomogeneous[:4, :] / triangulatedPointsHomogeneous[3, :]#divide by t #TODO deepcopy is needed? #dimension #lmfrod hna one pt #[0] #VIP
                    addedPt = [(selectedImgIdx,pt1),(imgIdx,pt2)]
        if maxSeparationAngle >= RAY_ANGLE_TH:
            track3D[trackIdx] = addedPt3D
            pts3D.append(addedPt3D) 
            viewPts.append(addedPt)
            usedTracks.append(trackIdx)
    #bundle adjustment#here
    print("image ",selectedImgIdx ,"--> Done")                                                    

In [ ]:
def write_proj_matrix():
    for i in range(len(imagesModels)):
        outputFile = open(datasetPath+"projection2/projection"+filesNames[i].split(".png")[0][-4:]+".txt",mode="w+")
        outputFile.write("CONTOUR\n")
        pString = ""
        for row in imagesModels[i]["projMat"]:
            for col in row:
                pString += str('{0:0.5f}'.format(col))+" "
            pString += "\n"
        outputFile.write(pString)
        outputFile.close()
        
write_proj_matrix()

In [ ]:
help(cv.decomposeProjectionMatrix)
"""
tracks : [] --> imgIdx,featureIdx
usedTracks: [] --> indices in tracks
traks3D: [] --> 3D point of each track in usedTracks
usedImagesIdx: [] --> indices in imageModes, indices for images already used "whose proj matrix is already calculated"
###
#input to bundle adjustment
pts3D: [] --> patches
viewPts = [pts3DIdx] [] --> (imgIdx, pt) #frames 
###

"""

<h2>Projection Matrix for initialization images BOOK 

In [ ]:
'''
def compute_P_from_essential(E):
    """ Computes the second camera matrix (assuming P1 = [I 0])
    from an essential matrix. Output is a list of four
    possible camera matrices. """
    # make sure E is rank 2
    U,S,V = linalg.svd(E)
    if linalg.det(np.dot(U,V))<0:
        V = -V
    E = np.dot(U,np.dot(diag([1,1,0]),V))
    # create matrices (Hartley p 258)
    Z = skew([0,0,-1])
    W = array([[0,-1,0],[1,0,0],[0,0,1]])
    # return all four solutions
    P2 = [vstack((np.dot(U,np.dot(W,V)).T,U[:,2])).T,
    vstack((np.dot(U,np.dot(W,V)).T,-U[:,2])).T,
    vstack((np.dot(U,np.dot(W.T,V)).T,U[:,2])).T,
    vstack((np.dot(U,np.dot(W.T,V)).T,-U[:,2])).T]
    return P2

def compute_essential_from_F(F):
    E = np.matmul(np.matmul(np.transpose(K),F),K)
    return E

def triangulate_point(x1,x2,P1,P2):
    """ Point pair triangulation from least squares solution. """
    M = zeros((6,6))
    M[:3,:4] = P1
    M[3:,:4] = P2
    M[:3,4] = -x1
    M[3:,5] = -x2
    U,S,V = linalg.svd(M)
    X = V[-1,:4]
    return X / X[3]

def triangulate(x1,x2,P1,P2):
    """ Two-view triangulation of points in x1,x2 (3*n homog. coordinates). """
    n = x1.shape[1]
    if x2.shape[1] != n:
        raise ValueError("Number of points don’t match.")
    X = [ triangulate_point(x1[:,i],x2[:,i],P1,P2) for i in range(n)]
    return array(X).T

#Test

# calibration
K = np.array([[2394,0,932],[0,2398,628],[0,0,1]])
#[I|0]
P1 = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])

# compute camera matrices (P2 will be list of four solutions)
P2 = compute_P_from_essential(E)
#From the list of camera matrices, we pick the one that has the most scene points
#in front of both cameras after triangulation.

# pick the solution with points in front of cameras
ind = 0
maxres = 0
for i in range(4):
# triangulate inliers and compute depth for each camera
    X = triangulate(x1n[:,inliers],x2n[:,inliers],P1,P2[i])
    d1 = dot(P1,X)[2]
    d2 = dot(P2[i],X)[2]
    if sum(d1>0)+sum(d2>0) > maxres:
        maxres = sum(d1>0)+sum(d2>0)
        ind = i
        infront = (d1>0) & (d2>0)
# triangulate inliers and remove points not in front of both cameras
X = triangulate(x1n[:,inliers],x2n[:,inliers],P1,P2[ind])
X = X[:,infront]
'''

<h2>Homography BOOK

In [ ]:
'''
class HomographyRansacModel(object):
    """ Class for testing homography fit with ransac.py from
    http://www.scipy.org/Cookbook/RANSAC"""
    def __init__(self,debug=False):
        self.debug = debug
    def fit(self, data):
        """ Fit homography to four selected correspondences. """
        # transpose to fit H_from_points()
        data = data.T
        # from points
        fp = data[:3,:4]
        # target points
        tp = data[3:,:4]
        # fit homography and return
        return H_from_points(fp,tp)
    def get_error( self, data, H):
        """ Apply homography to all correspondences,
        return error for each transformed point. """
        data = data.T
        # from points
        fp = data[:3]
        # target points
        tp = data[3:]
        # transform fp
        fp_transformed = dot(H,fp)
        # normalize hom. coordinates
        for i in range(3):
            fp_transformed[i] /= fp_transformed[2]
        # return error per point
        return sqrt( sum((tp-fp_transformed)**2,axis=0) )
'''

In [ ]:
'''
def H_from_ransac(fp,tp,model,maxiter=1000,match_theshold=10):
    """ Robust estimation of homography H from point
    correspondences using RANSAC (ransac.py from
    http://www.scipy.org/Cookbook/RANSAC).
    input: fp,tp (3*n arrays) points in hom. coordinates. """
    import ransactmp
    # group corresponding points
    data = vstack((fp,tp))
    # compute H and return
    H,ransac_data = ransac.ransac(data.T,model,4,maxiter,match_theshold,10,return_all=True)
    return H,ransac_data['inliers']
'''

In [ ]:
'''
def make_homog(points):
    """ Convert a set of points (dim*n array) to
    homogeneous coordinates. """
    return vstack((points,ones((1,points.shape[1]))))
# function to convert the matches to hom. points
def convert_points(j):
    ndx = matches[j].nonzero()[0]
    fp =  make_homog(l[j+1][ndx,:2].T)
    ndx2 = [int(matches[j][i]) for i in ndx]
    tp =  make_homog(l[j][ndx2,:2].T)
    return fp,tp
#matches de 4 points bs walla??

# estimate the homographies
model = HomographyRansacModel()
fp,tp = convert_points(1)
H_12, Hinliears = H_from_ransac(fp,tp,model)[0]
'''